# Zadanie 6
> Andrzej Kocielski, 09.12.2023
___

Wracamy do bazy danych MNIST, która jest publicznie dostępnym zbiorem danych składającym się z obrazów odręcznych pojedynczych cyfr. Każdy obraz ma kwadrat o wymiarach 28 na 28 pikseli, co oznacza, że każdy z nich ma łącznie 784 piksele. Użyjemy standardowego podziału zestawu danych, który wynosi 60 000 obrazów do trenowania naszego modelu i 10 000 do testowania naszego modelu. Celem jest rozpoznawanie cyfr pisanych odręcznie, co oznacza, że mamy do przewidzenia 10 klas (od 0 do 9). Naszym celem będzie wytrenowanie sieci neuronowej tak, aby osiągała mniej niż 1% błędu predykcji lub 99%+ dokładności.
Do zbudowania tej sieci neuronowej użyjemy bezpłatnego, opartego na chmurze, obsługującego procesor graficzny środowiska notebooka [Jupyter: Google Colab](https://colab.research.google.com/drive/1lgm4TmAIihDNLso2jooorbkC5bJXp4BV?authuser=1#scrollTo=fSyZvnYn2m0e). Zacznijmy od zaimportowania zestawu danych MNIST, który jest wygodnie dostarczany przez bibliotekę Keras.
Zaimportujemy również klasy i funkcje, których będziemy potrzeb.

Zaimportujemy również klasy i funkcje, których będziemy potrzebować:

In [ ]:
import keras.datasets 
import keras.models 
from keras.models import Sequential # model of ANN
from keras.layers import Dense, Dropout, Flatten, Activation # layers of ANN
from keras.layers import Conv2D, MaxPooling2D 
from keras import backend as K

# import matplotlib for visualization
import matplotlib.pyplot as plt

from keras.datasets import mnist

Następnie podzielimy dane między zestawy treningowe i testowe:

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Następnie zaimportujemy bibliotekę Matplotlib na potrzeby wizualizacji danych i wykreślimy pierwsze 2 obrazy w zestawie danych treningowych.

P4: Wyjaśnij znaczenie "Współdzielenia parametrów" i "Rzadkości połączeń" w CNN.

Odp (Andrzej Kocielski):
Współdzielenie wag – wspólny zestaw wag (+ bias) dla wszystkich lokalnych pól receptywnych. → w efekcie, warstwa konwolucji nauczy  się rozpoznawać pewną (losową) cechę w obrazie.

Rzadkość połączeń - w przeciwieństwie do "gęstych" warstw, gdzie każdy neuron jest połączony z każdym, w CNN ma zastosowanie agregacja cech (pooling), tak iż nie połączeń jest mniej. Przykładow dobierana jest największa wartość (funkcja max) lub suma pierw. kwadratowego (funkcja L2).

In [ ]:
# plot 2 images as gray scale
plt.figure(figure=(7,7))
plt.imshow(X_train[0], cmap=plt.cm.binary)
plt.colorbar()
plt.show()

Zaczniemy od ustawienia parametrów tuningu modelu w następujący sposób:

In [ ]:
batch_size = ???
num_classes =  ???
epochs = ???

Następnie spłaszczymy nasz zestaw danych z obrazów o strukturze 28 na 28 pikseli do wektora, który ma wartości wejściowe 784 pikseli.Najpierw wprowadzimy wymiary naszego obrazu.

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

Znormalizujemy również wartości pikseli od skali szarości, która mieści się w zakresie od 0 do 255, do zakresu od 0 do 1. Robimy to, dzieląc każdy z nich przez 255:

In [ ]:
if K.image_data_format() == 'channels_first': 
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols) 
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else: 
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1) 
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'training samples')
print(X_test.shape[0], 'testing samples')

Ponieważ jest to problem klasyfikacji wieloklasowej (0-9), spowoduje to konwersję wektorów klas na macierze klas binarnych.

Załóżmy na przykład, że cyfrą jest 2, przekonwertujemy wektor z [0 1 2 3 4 5 6 7 8 9] na [0 0 1 0 0 0 0 0 0 0 0 0]. Użyjemy wbudowanej funkcji, którą zapewnia Keras.utils.to_categorical()

In [ ]:
y_train = ???
y_test = ???

Zdefiniujemy teraz nasz model sieci neuronowej, który, jak wspomniano, będzie używał TensorFlow jako zaplecza modelu. Struktura naszej konwolucyjnej architektury sieci neuronowej jest zdefiniowana następująco:
- Pierwsza ukryta warstwa to warstwa konwolucyjna, która ma 32 mapy obiektów, każda o rozmiarze 3 x 3 i używamy wyprostowanej funkcji aktywacji liniowej - .Conv2Drelu
- Następnie dodajemy kolejną warstwę konwolucyjną z 64 mapami obiektów
- Dodajemy trzecią warstwę konwolucyjną ze 128 mapami obiektów
- Następnie dodajemy warstwę puli, która jest skonfigurowana z rozmiarem puli 2 x 2MaxPooling2D1

Następnie nakładamy warstwę regularyzacyjną, która jest ustawiona tak, aby losowo wykluczyć 25% neuronów w warstwie - służy to do zmniejszenia nadmiernego dopasowaniaDropout
Następnie przekształcamy macierz 2-wymiarową w wektor za pomocą - pozwala to na przetwarzanie naszego wyjścia przez w pełni połączone warstwyFlatten
Następnie dodajemy w pełni połączoną warstwę, która ma 128 neuronów i funkcję aktywacji ReLU
Następnie dodamy kolejną warstwę regularyzacji, aby zmniejszyć nadmierne dopasowanie, tym razem losowo wykluczamy 50% neuronów

Kończymy sieć neuronową warstwą wyjściową, która ma 10 neuronów - tyle samo, ile klas w naszym problemie klasyfikacyjnym i funkcję aktywacji softmax. Spowoduje to wyświetlenie przewidywania prawdopodobieństwa, że cyfra należy do każdej klasy.

In [ ]:
# import TensorFlow backend
import tensorflow as tf
# define our model
model = Sequential()
???
???
???
???
???

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation=Activation(tf.nn.softmax)))

Następnie skompilujemy nasz model 

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
		optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Teraz dopasujemy model.

In [ ]:

???



Na koniec ocenimy nasz model i wydrukujemy dokładność:

In [ ]:

???
???
???